In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import gluonnlp as nlp
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [3]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [4]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size=768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(),
                              attention_mask=attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [5]:
max_len = 64
batch_size = 64

In [6]:
test_data = pd.read_csv(r"C:\Users\kimminsung\OneDrive\PythonWorkspace\ratings_test.txt", sep='\t')

dataset_test = nlp.data.TSVDataset(r"C:\Users\kimminsung\OneDrive\PythonWorkspace\ratings_test.txt", field_indices=[1, 2], num_discard_samples=1)

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

predict_indices = []

using cached model


In [14]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _, max_indices = torch.max(out, 1)
    predict_indices = max_indices

###
#print(test_data)
for i, row in test_data.iterrows():
    print(row.document + " " + str(row.label) + " predict: " + str(int(predict_indices[i])))

100%|██████████| 782/782 [00:51<00:00, 15.09it/s]굳 ㅋ 1 predict: 0
GDNTOPCLASSINTHECLUB 0 predict: 1
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 0 predict: 0
지루하지는 않은데 완전 막장임... 돈주고 보기에는.... 0 predict: 1
3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? 0 predict: 1
음악이 주가 된, 최고의 음악영화 1 predict: 1
진정한 쓰레기 0 predict: 0
마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 0 predict: 0
갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 0 predict: 1
이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. 1 predict: 1
괜찮네요오랜만포켓몬스터잼밌어요 1 predict: 1
한국독립영화의 한계 그렇게 아버지가 된다와 비교됨 0 predict: 1
청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다. 1 predict: 0
눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다. 1 predict: 1
"스토리, 연출, 연기, 비주얼 등 영화의 기본 조차 안된 영화에 무슨 평을 해. 이런 영화 찍고도 김문옥 감독은 ""내가 영화 경력이 몇OO인데 조무래기들이 내 영화를 평론해?"" 같은 마인드에 빠져있겠지?" 0 predict: 1
소위 ㅈ문가라는 평점은 뭐냐? 1 predict: 1



IndexError: index 16 is out of bounds for dimension 0 with size 16